In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


In [ ]:
!pip install selenium beautifulsoup4

In [32]:
# Seleniumライブラリからwebdriverモジュールをインポート

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service   ##selenium 4.10
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import re
import time
from datetime import datetime

# HTMLから株価データを抽出する
def extract_stock_data(html):
  #response=requests.get(url)
  #html=response.text
  soup = BeautifulSoup(html, 'html.parser')
  tip=soup.find('div', class_='highcharts-tooltip')
  tip_td=tip.find_all("td")
#  print(tip_td)
  values = [datetime.strptime(tip_td[0].text, '%Y/%m/%d').date().strftime('%Y-%m-%d')]  # 日付格納
  for j in range(1,len(tip_td)-1):
    if re.findall("始値|高値|安値|終値|売買高", tip_td[j].text):
#          print(tip_td[j].get_text())
          values.append(tip_td[j].get_text())
#          print(values)
  return values

# Webページへアクセスし、データを抽出する
def get_stock_values(url,obj):
  act=ActionChains(obj)
  graph=obj.find_elements(By.CLASS_NAME,'highcharts-grid')[2]
  act.move_to_element(graph)
  act.move_by_offset(graph.rect['width'] // 2,0)
  act.perform()

  # グラフの横幅を取得
  graph_width = graph.rect['width']
#  graph_width=5
  tiplist=[]
  for i in range(0,graph_width):
    act.move_by_offset(-1,0).perform()
    html = obj.page_source.encode('utf-8')
    tmp_value=extract_stock_data(html)
    if tmp_value not in tiplist:
      tiplist.append(tmp_value)

  return tiplist






#  Chromeブラウザ起動オプションを指定
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Google Chrome用のブラウザドライバーインスタンスを作成
#chrome_driver = webdriver.Chrome(options=chrome_options)
service = Service(executable_path=r'/usr/bin/chromedriver') ##selenium 4.10.0への変更による##
chrome_driver=webdriver.Chrome(service=service,options=chrome_options)   ##selenium 4.10.0への変更による##

# 日経新聞にアクセス
url='https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month'
chrome_driver.get(url)

start_time = datetime.today()  # 開始時刻を取得
graph_tiplist=get_stock_values(url,chrome_driver)
end_time = datetime.today()  # 終了時刻を取得
dif = end_time - start_time  # 差分を計算
start_str=start_time.strftime('%Y年%m月%d日 %H:%M:%S')
end_str=end_time.strftime('%Y年%m月%d日 %H:%M:%S')


# ブラウザを閉じる
chrome_driver.quit()

print(f'開始:{start_str},終了{end_str},処理時間{dif}')
for tip in graph_tiplist:
    print(tip)



開始:2023年08月19日 09:38:39,終了2023年08月19日 09:43:51,処理時間0:05:12.253192
['2023-08-18', '始値: 31,321.26', '高値: 31,644.89', '安値: 31,275.25', '終値: 31,450.76']
['2023-08-17', '始値: 31,621.98', '高値: 31,704.08', '安値: 31,309.68', '終値: 31,626']
['2023-08-16', '始値: 31,965.58', '高値: 31,997.28', '安値: 31,766.82', '終値: 31,766.82']
['2023-08-15', '始値: 32,372.53', '高値: 32,403.93', '安値: 32,217.52', '終値: 32,238.89']
['2023-08-14', '始値: 32,456.72', '高値: 32,613.99', '安値: 32,031.54', '終値: 32,059.91']
['2023-08-10', '始値: 32,015.96', '高値: 32,504.85', '安値: 32,015.96', '終値: 32,473.65']
['2023-08-09', '始値: 32,346.32', '高値: 32,407.85', '安値: 32,175.64', '終値: 32,204.33']
['2023-08-08', '始値: 32,430.61', '高値: 32,539.88', '安値: 32,238.59', '終値: 32,377.29']
['2023-08-07', '始値: 31,921.28', '高値: 32,306.87', '安値: 31,830.23', '終値: 32,254.56']
['2023-08-04', '始値: 32,019.06', '高値: 32,294.7', '安値: 31,934.35', '終値: 32,192.75']
['2023-08-03', '始値: 32,375.85', '高値: 32,467.06', '安値: 32,142.25', '終値: 32,159.28']
['2023-08-02', '始値: 33,12